In [2]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import scipy.sparse as sparse

In [4]:
import numpy as np
import pandas as pd

In [5]:
from thesislib.utils import pathutils

In [6]:
sparse_csv = pathutils.get_data_file("04_06_new_data/sparse/symptoms_04_16_4.csv")

In [7]:
df = pd.read_csv(sparse_csv, index_col='Index')

In [8]:
labels = df.LABEL
symptoms = df.SYMPTOMS
df = df.drop(columns=['LABEL', 'SYMPTOMS'])

In [9]:
dense_matrix = sparse.coo_matrix(df.values)

In [10]:
symptoms = symptoms.apply(lambda v: [int(idx) for idx in v.split(",")])

In [11]:
columns = []
rows = []
for idx, val in symptoms.iteritems():
    rows += [idx] * len(val)
    columns += val

In [12]:
data = np.ones(len(rows))

In [13]:
num_symptoms = 376

In [14]:
symptoms_coo = sparse.coo_matrix((data, (rows, columns)), shape=(df.shape[0], num_symptoms))

In [15]:
data_coo = sparse.hstack([dense_matrix, symptoms_coo])

In [80]:
df.columns

Index(['GENDER', 'RACE', 'AGE'], dtype='object')

In [16]:
data_csc = data_coo.tocsc()

In [81]:
data_csc.shape

(107921, 379)

In [79]:
kk.shape

(107921, 1)

In [17]:
from sklearn.model_selection import StratifiedShuffleSplit

In [18]:
label_values = labels.values

In [19]:
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(data_csc, label_values):
    train_data = data_csc[train_index]
    train_labels = label_values[train_index]
    test_data = data_csc[test_index]
    test_labels = label_values[test_index]

In [20]:
train_data.shape

(86336, 379)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
clf = RandomForestClassifier(n_estimators=140, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=2, random_state=None, verbose=0, warm_start=False, class_weight=None)

In [23]:
clf = clf.fit(train_data, train_labels)

In [24]:
from sklearn.metrics import accuracy_score, make_scorer

In [25]:
accuracy_scorer = make_scorer(accuracy_score)

In [26]:
train_score = accuracy_scorer(clf, train_data, train_labels)

In [27]:
test_score = accuracy_scorer(clf, test_data, test_labels)

In [58]:
from thesislib.utils.ml import report

In [71]:
top_5_scorer = make_scorer(report.top_n_score, needs_proba=True, class_labels=classes, top_n=2)

In [72]:
train_top_5_score = top_5_scorer(clf, train_data, train_labels)

In [73]:
test_top_5_score = top_5_scorer(clf, test_data, test_labels)

In [74]:
print("Train Score: %.3f\n Test Score: %.3f" % (train_score, test_score))

Train Score: 0.945
 Test Score: 0.856


In [75]:
print("Top 5 Train Score: %.3f\nTop 5 Test Score: %.3f" % (train_top_5_score, test_top_5_score))

Top 5 Train Score: 0.998
Top 5 Test Score: 0.991


In [29]:
import pandas as pd

In [30]:
classes = np.unique(test_labels)

In [31]:
from sklearn.preprocessing import LabelEncoder
labelbin = LabelEncoder()
labelbin.fit(classes)

LabelEncoder()

In [32]:
test_prob = clf.predict_proba(test_data)

In [33]:
sorted_prob = np.argsort(-test_prob, axis=1)

In [34]:
encoded_labels = labelbin.transform(test_labels)

In [50]:
top_five_sorted = sorted_prob[:, :5]

In [51]:
encoded_probability = np.take_along_axis(test_prob, encoded_labels[:, None], axis=1)

In [52]:
encoded_probability = encoded_probability.reshape(encoded_probability.shape[0],)

In [53]:
print(top_five_sorted[:5])

[[6 0 1 2 3]
 [2 7 0 1 3]
 [2 7 0 1 3]
 [4 0 1 2 3]
 [2 7 0 1 3]]


In [54]:
print(encoded_labels[:5])

[6 7 2 4 7]


In [42]:
res = np.zeros((3, ), dtype=bool)

In [1]:
def check_is_in(needles, haystack):
    if needles.shape[0] != haystack.shape[0]:
        raise ValueError("Needles and Haystack shape mismatch")

    result = np.zeros((needles.shape[0], ), dtype=bool)

    for idx in range(haystack.shape[0]):
        result[idx] = np.isin(needles[idx], haystack[idx, :]).reshape(1, )[0]
    
    return result

In [55]:
bool_top_five = check_is_in(encoded_labels, top_five_sorted)

In [ ]:
bool_top_five = bool_top_five[:, 0]

In [ ]:
bool_top_five[:5]

In [56]:
combined = np.logical_and(bool_top_five, (encoded_probability > 0))

In [57]:
sum(combined)/combined.shape[0]

0.9963400509613157

In [82]:
from thesislib.utils.ml import models

In [84]:
nb_df = pd.read_csv(sparse_csv, index_col="Index")

In [86]:
nb_labels = nb_df.LABEL
nb_df = nb_df.drop(columns=['LABEL'])

In [103]:
nb_labels = nb_labels.values

In [93]:
sparsifier = models.ThesisSymptomSparseMaker(num_symptoms=num_symptoms)
nb_data = sparsifier.fit_transform(nb_df)

In [105]:
nb_train_data = nb_data[train_index]
nb_train_labels = nb_labels[train_index]
nb_test_data = nb_data[test_index]
nb_test_labels = nb_labels[test_index]

In [113]:
import importlib
_ = importlib.reload(models)

In [94]:
nb_data.shape

(107921, 379)

In [95]:
nb_df.columns

Index(['GENDER', 'RACE', 'AGE', 'SYMPTOMS'], dtype='object')

In [96]:
from sklearn import naive_bayes

In [114]:
symptom_clf = naive_bayes.BernoulliNB()
gender_clf = naive_bayes.BernoulliNB()
race_clf = naive_bayes.MultinomialNB()
age_clf = naive_bayes.GaussianNB()

In [115]:
classifier_map = [
    [gender_clf, [0, False]],
    [race_clf, [1, False]],
    [age_clf, [2, False]],
    [symptom_clf, [(3, None), True]],
]

In [116]:
clf = models.ThesisSparseNaiveBayes(classifier_map=classifier_map, classes=classes)

In [117]:
clf.fit(nb_train_data, nb_train_labels)

In [118]:
train_score = accuracy_scorer(clf, nb_train_data, nb_train_labels)

In [119]:
test_score = accuracy_scorer(clf, nb_test_data, nb_test_labels)

In [58]:
from thesislib.utils.ml import report

In [120]:
top_5_scorer = make_scorer(report.top_n_score, needs_proba=True, class_labels=classes, top_n=5)

In [72]:
train_top_5_score = top_5_scorer(clf, nb_train_data, nb_train_labels)

In [121]:
test_top_5_score = top_5_scorer(clf, nb_test_data, nb_test_labels)

In [122]:
print("Train Score: %.3f\n Test Score: %.3f" % (train_score, test_score))
print("Top 5 Train Score: %.3f\nTop 5 Test Score: %.3f" % (train_top_5_score, test_top_5_score))

Train Score: 0.849
 Test Score: 0.848
Top 5 Train Score: 0.998
Top 5 Test Score: 1.000


Top 5 Train Score: 0.998
Top 5 Test Score: 0.991
